# Importing the European Roads dataset

In this notebook we're going to import the European Roads dataset into Neo4j. Let's first import the Neo4j Python driver:

In [1]:
pip install neo4j sklearn altair

  Using cached https://files.pythonhosted.org/packages/01/55/0bb2226e34f21fa549c3f4557b4f154a5632f61132a969da17c95ca8eab9/altair-4.1.0-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [3]:
from neo4j import GraphDatabase
from neo4j.exceptions import ClientError
import os

Once we've done that we can initialise the Neo4j driver.

In [4]:
bolt_url = os.getenv("NEO4J_BOLT_URL", "bolt://localhost")
user = os.getenv("NEO4J_USER", "neo4j")
password = os.getenv("NEO4J_PASSWORD", "neo")
driver = GraphDatabase.driver("bolt://graph-embeddings-neo4j", auth=(user, password))

## Importing the dataset

And now we can import the dataset:

In [6]:
with driver.session(database="neo4j") as session:
    try: 
        session.run("""
        CREATE CONSTRAINT ON (p:Place) ASSERT p.name IS UNIQUE;
        """)
    except ClientError as ex:
        print(ex)
        
    result = session.run("""
    USING PERIODIC COMMIT 1000
    LOAD CSV WITH HEADERS FROM "https://github.com/neo4j-examples/graph-embeddings/raw/main/data/roads.csv"
    AS row

    MERGE (origin:Place {name: row.origin_reference_place})
    MERGE (originCountry:Country {code: row.origin_country_code})

    MERGE (destination:Place {name: row.destination_reference_place})
    MERGE (destinationCountry:Country {code: row.destination_country_code})
    
    MERGE (origin)-[:IN_COUNTRY]->(originCountry)
    MERGE (destination)-[:IN_COUNTRY]->(destinationCountry)

    MERGE (origin)-[eroad:EROAD {number: row.road_number}]->(destination)
    SET eroad.distance = toInteger(row.distance), eroad.watercrossing = row.watercrossing;
    """)
    display(result.consume().counters)

An equivalent constraint already exists, 'Constraint( UNIQUE, :Place(name) )'.


{'labels_added': 933, 'relationships_created': 2145, 'nodes_created': 933, 'properties_set': 4683}